### Import libraries

First of all, we import all the required modules and classes from the specific libraries here to centralize the imports at a single place.

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch import FloatTensor, LongTensor
from torch.utils.data import DataLoader, TensorDataset

### Select Device

Here, we select the device to be cuda is it is available, and use cpu for fallback.

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Define Constants

Here, we define all the required constants to centralize the constant values at a single place.

In [8]:
SAVED_DATASETS_DIR = '../../saved/processed/'
BATCH_SIZE = 128
EPOCHS = 20
LEARNING_RATE = 0.01
DROPOUT_RATE = 0.3

### Load Preprocessed Datasets

Now, we load the saved preprocessed datasets for usage, and check the shape for each dataset.

In [13]:
X_train = np.load(SAVED_DATASETS_DIR + 'X_train.npy')
X_val = np.load(SAVED_DATASETS_DIR + 'X_val.npy')
y_train = np.load(SAVED_DATASETS_DIR + 'y_train.npy')
y_val = np.load(SAVED_DATASETS_DIR + 'y_val.npy')

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape: ', y_train.shape)
print('y_val shape: ', y_val.shape)

X_train shape:  (17784, 256)
X_val shape:  (1976, 256)
y_train shape:  (17784,)
y_val shape:  (1976,)


### Create TensorDataset and DataLoader

Now, we use the loaded datasets, convert them into tensors, create the tensor datasets, create the data loaders, and take the input and output dimensions of the data in order to use them for training the model.

In [16]:
X_train_tensor = FloatTensor(X_train)
X_val_tensor = FloatTensor(X_val)
y_train_tensor = LongTensor(y_train)
y_val_tensor = LongTensor(y_val)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

input_dim = X_train_tensor.shape[1]
output_dim = len(y_train_tensor.unique())
print(f"Input = {input_dim}, Output = {output_dim}")

Input = 256, Output = 17


### Define Neural Network Model

Now, we build a simple feedforward neural network for multi-class currency classification and initialize the model into the gpu(cuda).

In [17]:
class CurrencyClassifier(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate):
        super().__init__()

        self.fc1 = nn.Linear(input_size, 192)
        self.bn1 = nn.BatchNorm1d(192)

        self.fc2 = nn.Linear(192, 96)
        self.bn2 = nn.BatchNorm1d(96)

        self.fc3 = nn.Linear(96, 48)
        self.bn3 = nn.BatchNorm1d(48)

        self.fc4 = nn.Linear(48, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.dropout(self.relu(self.bn1(self.fc1(x))))
        x = self.dropout(self.relu(self.bn2(self.fc2(x))))
        x = self.dropout(self.relu(self.bn3(self.fc3(x))))
        x = self.fc4(x)

        return x

model = CurrencyClassifier(input_dim, output_dim, DROPOUT_RATE).to(device)
print(model)

CurrencyClassifier(
  (fc1): Linear(in_features=256, out_features=192, bias=True)
  (bn1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=192, out_features=96, bias=True)
  (bn2): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=96, out_features=48, bias=True)
  (bn3): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=48, out_features=17, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)
